In [1]:
#import necessary libraries
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import re

import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [2]:
#json_file_path = input('Please upload your json file for test')

In [3]:
#function to read the jason file

def preprocess_dataset(json_file_path):
    df_train = pd.read_json(json_file_path) #read json
    df_train = df_train.drop_duplicates() #drop duplicates in dataset
    df_train_fr = df_train[df_train.culture != 'nl-nl'] #select only the france in text, as the dutch is much less
    df_train = df_train_fr[df_train_fr.source != 'WORKFLOW']
    df_train = df_train.drop(columns = ['culture','document_id','annotation_id','source'])
    return df_train

In [4]:
df_train = preprocess_dataset("./elevator_data/kone_classification.json")

In [5]:
df_train.head()

,text,label
0,"Eclairage de la gaine , à compléter rappel : 2",SH16
4,Le boitier métallique sur le toit de cabine es...,CAR23
5,Plaque d ' identification n ' est pas complète...,CAR26
6,Le bon fonctionnement du système appel de seco...,CAR01
8,Les données du propriétaire sont à placer en c...,CAR06


## French text cleaning

In [6]:
#!pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git &> /dev/null

In [7]:
import nltk
import string
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from nltk import word_tokenize

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yihuiwang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/yihuiwang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yihuiwang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
french_stopwords = nltk.corpus.stopwords.words('french')
lemmatizer = FrenchLefffLemmatizer()

In [9]:
def French_Preprocess(text):
    final_string = ""

    
    sentence_w_punct = "".join([i.lower() for i in text if i not in string.punctuation])

    sentence_w_num = ''.join(i for i in sentence_w_punct if not i.isdigit())

    tokenize_sentence = nltk.tokenize.word_tokenize(sentence_w_num)

    words_w_stopwords = [i for i in tokenize_sentence if i not in french_stopwords]

    words_lemmatize = (lemmatizer.lemmatize(w) for w in words_w_stopwords)

    final_string = ' '.join(w for w in words_lemmatize)

    return final_string

In [10]:
df_train['text']

0         Eclairage de la gaine , à compléter rappel : 2
4      Le boitier métallique sur le toit de cabine es...
5      Plaque d ' identification n ' est pas complète...
6      Le bon fonctionnement du système appel de seco...
8      Les données du propriétaire sont à placer en c...
                             ...                        
517    Dossier de sécurité : le rapport d ' analyse d...
518             La vitre de la boîte à clé est absente .
519                   Nettoyer la cuvette 1 e Répétition
523    Veuillez fixer les 3 poids dont la taille ne p...
524    Dossier de sécurité : les documents concernant...
Name: text, Length: 182, dtype: object

In [11]:
df_train['text'] = df_train['text'].apply(lambda x: French_Preprocess((x)))

In [12]:
df_train.head()

,text,label
0,eclairage gaine compléter rappel,SH16
4,boitier métallique toit cabine refixer correct...,CAR23
5,plaque identification complète correcte charge...,CAR26
6,bon fonctionnement système appel secours cabin...,CAR01
8,donnée propriétaire placer cabine,CAR06


In [13]:
# Create a new column 'category_id' with encoded categories 
df_train['category_id'] = df_train['label'].factorize()[0]
df_train.head()

,text,label,category_id
0,eclairage gaine compléter rappel,SH16,0
4,boitier métallique toit cabine refixer correct...,CAR23,1
5,plaque identification complète correcte charge...,CAR26,2
6,bon fonctionnement système appel secours cabin...,CAR01,3
8,donnée propriétaire placer cabine,CAR06,4


In [14]:
df_train.to_csv('clean_data.csv', index=False)